## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-29-21-51-03 +0000,nyt,"Tran Trong Duyet, John McCain’s Captor at the ...",https://www.nytimes.com/2025/08/29/world/asia/...
1,2025-08-29-21-51-00 +0000,wsj,Opinion | The Failing Political Lords of Lords...,https://www.wsj.com/opinion/lordstown-foxconn-...
2,2025-08-29-21-50-00 +0000,wsj,Opinion | The U.N.’s Days in Lebanon Are Numbered,https://www.wsj.com/opinion/unifil-lebanon-tru...
3,2025-08-29-21-49-37 +0000,nyt,"Riding Amtrak’s New Acela Train: Sleeker, Comf...",https://www.nytimes.com/2025/08/29/travel/amtr...
4,2025-08-29-21-48-51 +0000,nypost,Ukraine deploys drones to blow up Moscow’s hid...,https://nypost.com/2025/08/29/world-news/ukrai...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2306/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
40,trump,53
18,new,26
116,will,14
163,shooting,14
281,minneapolis,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
67,2025-08-29-19-45-00 +0000,wsj,A federal judge signaled that she will move qu...,https://www.wsj.com/politics/policy/judge-weig...,127
11,2025-08-29-21-39-00 +0000,wsj,A federal appeals court late Friday struck dow...,https://www.wsj.com/politics/policy/appeals-co...,113
68,2025-08-29-19-42-00 +0000,wsj,Republican Sen. Joni Ernst isn’t expected to s...,https://www.wsj.com/politics/elections/republi...,102
253,2025-08-29-07-56-31 +0000,wsj,Stock Market Today: Appeals Court Strikes Down...,https://www.wsj.com/livecoverage/stock-market-...,97
193,2025-08-29-12-34-51 +0000,bbc,Fed Governor Cook in court to block Trump from...,https://www.bbc.com/news/articles/cedvj2d5538o...,96


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
67,127,2025-08-29-19-45-00 +0000,wsj,A federal judge signaled that she will move qu...,https://www.wsj.com/politics/policy/judge-weig...
310,67,2025-08-29-00-39-00 +0000,wsj,New Trial Ordered for Three Former Police Offi...,https://www.wsj.com/us-news/law/new-trial-orde...
11,52,2025-08-29-21-39-00 +0000,wsj,A federal appeals court late Friday struck dow...,https://www.wsj.com/politics/policy/appeals-co...
169,47,2025-08-29-14-03-22 +0000,nypost,California Gov. Newsom says crime is higher in...,https://nypost.com/2025/08/29/us-news/californ...
38,45,2025-08-29-20-47-01 +0000,nypost,Rosie O’Donnell falsely rants that transgender...,https://nypost.com/2025/08/29/us-news/rosie-od...
20,45,2025-08-29-21-28-49 +0000,nypost,15-year-old NYC boy with long rap sheet and an...,https://nypost.com/2025/08/29/us-news/15-year-...
271,37,2025-08-29-04-27-33 +0000,nypost,JD Vance blasts Jen Psaki for post declaring p...,https://nypost.com/2025/08/29/us-news/jd-vance...
237,35,2025-08-29-09-09-26 +0000,cbc,South Korea sees high-profile indictments of f...,https://www.cbc.ca/news/world/south-korea-form...
68,35,2025-08-29-19-42-00 +0000,wsj,Republican Sen. Joni Ernst isn’t expected to s...,https://www.wsj.com/politics/elections/republi...
188,34,2025-08-29-12-49-17 +0000,nypost,Fed’s preferred inflation gauge stays flat — b...,https://nypost.com/2025/08/29/business/feds-pr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
